In [ ]:
%matplotlib inline

import logging
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
import sys
sys.path.append('/Users/urebbapr/research/opticalcomm/MLOC/src')

from functools          import partial
from matplotlib.patches import Patch
from matplotlib.ticker  import FormatStrFormatter
from types              import SimpleNamespace as SN

# Depending on how your environment is setup, may need to tweak the utils import
from utils import (
    load_pwv,
    compile_datasets
)

# Disable warnings
warnings.filterwarnings('ignore')

# Disable logger except for errors/exceptions
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

# Set context of seaborn
sns.set_context('talk')


In [ ]:
def compile_pwv(path, h5=None, resample='30 min'):
    """
    Ingests all of the raw data into dataframes then compiles them into a merged
    dataframe.

    Parameters
    ----------
    h5 : str
        Optional path to an h5 to write to
    resample : str
        The rate to resample the merged dataframe
    smooth : list of str
        List of columns to apply smoothing on
    """

    logger.debug('Loading datasets from their raw files')
    # Load in the data
    df = load_pwv(path)

    # Save individual frames
    if h5 is not None:
        df.to_hdf(h5, 'pwv')

    # Sort the datetime index
    df.sort_index(inplace=True)

    logger.debug(f'Resampling to {resample}')
    df = df.resample(resample).median()#.apply(minobs)

    # Save merged
    df.to_hdf(h5, 'pwv')

    return df

In [ ]:
df = compile_pwv('/Users/urebbapr/research/opticalcomm/data/v3/PVWwithWeather.txt', 'pwv.h5')

In [ ]:
df.describe()

In [ ]:
a = pd.read_hdf('pwv.h5')
a.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(7, 1, figsize=(30,35))
for i, year in enumerate(range(2009, 2016)):
    ax[i].set_title(year)
    mask = (a.index >= '{}-01-01'.format(year)) & (a.index <= '{}-12-31'.format(year))
    ax[i].plot(df.loc[mask].index, df.loc[mask].water_vapor, 'r.')